<a href="https://colab.research.google.com/github/ErickNicolasP/ASE/blob/main/proyecto_ASE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CARGAMOS TODOS LOS DATOS NECESARIOS, BIBLIOTECAS Y ACOMODAMOS LA INFORMACIÓN PARA SU PROXIMA MANIPULACIÓN

In [14]:
#Cargamos el archivo desde google drive
from google.colab import drive
from google.colab import files
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [15]:
#importamos las librerías
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
import io as io

In [16]:
#cargamos los data frame en las variables
#2018
dev_2018 = pd.read_csv('/content/gdrive/MyDrive/Ciencia de Datos/ASE/2018/DEV 2018.csv')
#2019
dev_2019 = pd.read_csv('/content/gdrive/MyDrive/Ciencia de Datos/ASE/2019/DEV 2019.csv')
#2020
dev_2020_enero = pd.read_csv('/content/gdrive/MyDrive/Ciencia de Datos/ASE/2020/DEV 2020_enero-marzo.csv')
dev_2020_julio = pd.read_csv('/content/gdrive/MyDrive/Ciencia de Datos/ASE/2020/DEV 2020_julio-septiembre.csv')
dev_2020_octubre = pd.read_csv('/content/gdrive/MyDrive/Ciencia de Datos/ASE/2020/DEV 2020_octubre-diciembre.csv')
#2022
dev_2022_julio = pd.read_csv('/content/gdrive/MyDrive/Ciencia de Datos/ASE/2022/DEV Julio-Septiembre 2022.csv')
dev_2022_octubre = pd.read_csv('/content/gdrive/MyDrive/Ciencia de Datos/ASE/2022/DEV Octubre-Diciembre 2022.csv')

#poblacion 2020, esta se usará de referencia para sacar la tasa_2018, ya que el censo se hace cada 10 años, el más acertado es el de 2020
censo = pd.read_csv('/content/gdrive/MyDrive/Ciencia de Datos/ASE/censo 2020/censo 2020.csv')

<ipython-input-16-fde523059dcc>:3: DtypeWarning: Columns (496,497) have mixed types. Specify dtype option on import or set low_memory=False.
  dev_2018 = pd.read_csv('/content/gdrive/MyDrive/Ciencia de Datos/ASE/2018/DEV 2018.csv')
<ipython-input-16-fde523059dcc>:5: DtypeWarning: Columns (494,496) have mixed types. Specify dtype option on import or set low_memory=False.
  dev_2019 = pd.read_csv('/content/gdrive/MyDrive/Ciencia de Datos/ASE/2019/DEV 2019.csv')


In [17]:
#Los años 2020 y 2022, no tienen los data frame concatenados, por lo que eso se hará acontinuación
#2020
dev_2020 = pd.concat([dev_2020_enero,dev_2020_julio,dev_2020_octubre], axis=0)
#2022
dev_2022 = pd.concat([dev_2022_julio,dev_2022_octubre],axis=0)

#variables de los data frames
#----dev_2018
#----dev_2019
#----dev_2020
#----dev_2022



#EDAD Y EL SEXO CON SU TASA DE MIGRACIÓN. 2018


In [20]:
hombres_df_2018 = dev_2018.loc[(dev_2018["p1"] == 1) & (dev_2018["p2"] < 97), ["p2", "p1"]].value_counts().reset_index()
hombres_df_2018 = hombres_df_2018.loc[:, ["p2", 0]]
hombres_df_2018.columns = ["edad", "contar"]

mujeres_df_2018 = dev_2018.loc[(dev_2018["p1"] == 2) & (dev_2018["p2"] < 97), ["p2", "p1"]].value_counts().reset_index()
mujeres_df_2018 = mujeres_df_2018.loc[:, ["p2", 0]]
mujeres_df_2018.columns = ["edad", "contar"]

In [21]:
if hombres_df_2018["edad"].max() > mujeres_df_2018["edad"].max() :
    max = hombres_df_2018["edad"].max()
else:
    max = mujeres_df_2018["edad"].max()

In [22]:
rango_edades = set(range(0, max+1))
edades_presentes = set(hombres_df_2018['edad'])
edades_faltantes = rango_edades - edades_presentes

for edad in edades_faltantes:
    hombres_df_2018.loc[len(hombres_df_2018)] = [edad, 0]
hombres_df_2018 = hombres_df_2018.sort_values("edad").reset_index(drop=True)
hombres_df_2018

,edad,contar
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
67,67,0
68,68,0
69,69,0
70,70,0


In [23]:
rango_edades = set(range(0, max+1))
edades_presentes = set(mujeres_df_2018['edad'])
edades_faltantes = rango_edades - edades_presentes

for edad in edades_faltantes:
    mujeres_df_2018.loc[len(mujeres_df_2018)] = [edad, 0]
mujeres_df_2018 = mujeres_df_2018.sort_values("edad").reset_index(drop=True)
mujeres_df_2018

,edad,contar
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
67,67,0
68,68,1
69,69,0
70,70,0


In [24]:
censo_2018 = censo
tasa_2018 = pd.DataFrame(columns=['hombres', 'mujeres'])
censo_2018 = censo_2018.replace(',', '', regex=True).astype('int64')
censo_2018 = censo_2018.iloc[0:max+1]

In [25]:
tasa_2018["hombres"] = (hombres_df_2018["contar"]/censo_2018["Hombres"])*1000
tasa_2018["mujeres"] = (mujeres_df_2018["contar"]/censo_2018["Mujeres"])*1000
tasa_2018

,hombres,mujeres
0,0.00000,0.000000
1,0.00000,0.000000
2,0.00000,0.000000
3,0.00000,0.000000
4,0.00000,0.000000
...,...,...
67,0.00000,0.000000
68,0.00000,0.002474
69,0.00000,0.000000
70,0.00000,0.000000


# EDAD Y SEXO CON SU TASA DE MIGRACIÓN. 2019

In [26]:
hombres_df_2019 = dev_2019.loc[(dev_2019["p1"] == 1) & (dev_2019["p2"] < 97), ["p2", "p1"]].value_counts().reset_index()
hombres_df_2019 = hombres_df_2019.loc[:, ["p2", 0]]
hombres_df_2019.columns = ["edad", "contar"]

mujeres_df_2019 = dev_2019.loc[(dev_2019["p1"] == 2) & (dev_2019["p2"] < 97), ["p2", "p1"]].value_counts().reset_index()
mujeres_df_2019 = mujeres_df_2019.loc[:, ["p2", 0]]
mujeres_df_2019.columns = ["edad", "contar"]

In [27]:
if hombres_df_2019["edad"].max() > mujeres_df_2019["edad"].max() :
    max = hombres_df_2019["edad"].max()
else:
    max = mujeres_df_2019["edad"].max()

In [28]:
rango_edades = set(range(0, max+1))
edades_presentes = set(hombres_df_2019['edad'])
edades_faltantes = rango_edades - edades_presentes

for edad in edades_faltantes:
    hombres_df_2019.loc[len(hombres_df_2019)] = [edad, 0]
hombres_df_2019 = hombres_df_2019.sort_values("edad").reset_index(drop=True)
hombres_df_2019

,edad,contar
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
71,71,0
72,72,0
73,73,0
74,74,1


In [29]:
rango_edades = set(range(0, max+1))
edades_presentes = set(mujeres_df_2019['edad'])
edades_faltantes = rango_edades - edades_presentes

for edad in edades_faltantes:
    mujeres_df_2019.loc[len(mujeres_df_2019)] = [edad, 0]
mujeres_df_2019 = mujeres_df_2019.sort_values("edad").reset_index(drop=True)
mujeres_df_2019

,edad,contar
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
71,71,0
72,72,0
73,73,0
74,74,0


In [30]:
censo_2019 = censo
tasa_2019 = pd.DataFrame(columns=['hombres', 'mujeres'])
censo_2019 = censo_2019.replace(',', '', regex=True).astype('int64')
censo_2019 = censo_2019.iloc[0:max+1]

In [31]:
tasa_2019["hombres"] = (hombres_df_2019["contar"]/censo_2019["Hombres"])*1000
tasa_2019["mujeres"] = (mujeres_df_2019["contar"]/censo_2019["Mujeres"])*1000
tasa_2019

,hombres,mujeres
0,0.000000,0.0
1,0.000000,0.0
2,0.000000,0.0
3,0.000000,0.0
4,0.000000,0.0
...,...,...
71,0.000000,0.0
72,0.000000,0.0
73,0.000000,0.0
74,0.004757,0.0


#EDAD Y SEXO CON SU TASA DE MIGRACIÓN. 2020

In [32]:
hombres_df_2020 = dev_2020.loc[(dev_2020["a1"] == 1) & (dev_2020["a2"] < 97), ["a2", "a1"]].value_counts().reset_index()
hombres_df_2020 = hombres_df_2020.loc[:, ["a2", 0]]
hombres_df_2020.columns = ["edad", "contar"]

mujeres_df_2020 = dev_2020.loc[(dev_2020["a1"] == 2) & (dev_2020["a2"] < 97), ["a2", "a1"]].value_counts().reset_index()
mujeres_df_2020 = mujeres_df_2020.loc[:, ["a2", 0]]
mujeres_df_2020.columns = ["edad", "contar"]

In [33]:
if hombres_df_2020["edad"].max() > mujeres_df_2020["edad"].max() :
    max = hombres_df_2020["edad"].max()
else:
    max = mujeres_df_2020["edad"].max()

In [34]:
rango_edades = set(range(0, max+1))
edades_presentes = set(hombres_df_2020['edad'])
edades_faltantes = rango_edades - edades_presentes

for edad in edades_faltantes:
    hombres_df_2020.loc[len(hombres_df_2020)] = [edad, 0]
hombres_df_2020 = hombres_df_2020.sort_values("edad").reset_index(drop=True)
hombres_df_2020

,edad,contar
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
61,61,1
62,62,5
63,63,3
64,64,0


In [35]:
rango_edades = set(range(0, max+1))
edades_presentes = set(mujeres_df_2020['edad'])
edades_faltantes = rango_edades - edades_presentes

for edad in edades_faltantes:
    mujeres_df_2020.loc[len(mujeres_df_2020)] = [edad, 0]
mujeres_df_2020 = mujeres_df_2020.sort_values("edad").reset_index(drop=True)
mujeres_df_2020

,edad,contar
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
61,61,0
62,62,0
63,63,0
64,64,0


In [36]:
censo_2020 = censo
tasa_2020 = pd.DataFrame(columns=['hombres', 'mujeres'])
censo_2020 = censo_2020.replace(',', '', regex=True).astype('int64')
censo_2020 = censo_2020.iloc[0:max+1]

In [37]:
tasa_2020["hombres"] = (hombres_df_2020["contar"]/censo_2020["Hombres"])*1000
tasa_2020["mujeres"] = (mujeres_df_2020["contar"]/censo_2020["Mujeres"])*1000
tasa_2020

,hombres,mujeres
0,0.000000,0.0
1,0.000000,0.0
2,0.000000,0.0
3,0.000000,0.0
4,0.000000,0.0
...,...,...
61,0.002881,0.0
62,0.010947,0.0
63,0.006954,0.0
64,0.000000,0.0


#EDAD Y SEXO CON SU TASA DE MIGRACIÓN 2022



In [40]:
hombres_df_2022 = dev_2022.loc[(dev_2022["a1"] == 1) & (dev_2022["a2"] < 97), ["a2", "a1"]].value_counts().reset_index()
hombres_df_2022 = hombres_df_2022.loc[:, ["a2", 0]]
hombres_df_2022.columns = ["edad", "contar"]

mujeres_df_2022 = dev_2022.loc[(dev_2022["a1"] == 2) & (dev_2022["a2"] < 97), ["a2", "a1"]].value_counts().reset_index()
mujeres_df_2022 = mujeres_df_2022.loc[:, ["a2", 0]]
mujeres_df_2022.columns = ["edad", "contar"]

In [41]:
if hombres_df_2022["edad"].max() > mujeres_df_2022["edad"].max() :
    max = hombres_df_2022["edad"].max()
else:
    max = mujeres_df_2022["edad"].max()

In [42]:
rango_edades = set(range(0, max+1))
edades_presentes = set(hombres_df_2022['edad'])
edades_faltantes = rango_edades - edades_presentes

for edad in edades_faltantes:
    hombres_df_2022.loc[len(hombres_df_2022)] = [edad, 0]
hombres_df_2022 = hombres_df_2022.sort_values("edad").reset_index(drop=True)
hombres_df_2022

,edad,contar
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
76,76,0
77,77,0
78,78,0
79,79,0


In [43]:
rango_edades = set(range(0, max+1))
edades_presentes = set(mujeres_df_2022['edad'])
edades_faltantes = rango_edades - edades_presentes

for edad in edades_faltantes:
    mujeres_df_2022.loc[len(mujeres_df_2022)] = [edad, 0]
mujeres_df_2022 = mujeres_df_2022.sort_values("edad").reset_index(drop=True)
mujeres_df_2022

,edad,contar
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
76,76,0
77,77,0
78,78,0
79,79,0


In [44]:
censo_2022 = censo
tasa_2022 = pd.DataFrame(columns=['hombres', 'mujeres'])
censo_2022 = censo_2022.replace(',', '', regex=True).astype('int64')
censo_2022 = censo_2022.iloc[0:max+1]

In [45]:
tasa_2022["hombres"] = (hombres_df_2022["contar"]/censo_2022["Hombres"])*1000
tasa_2022["mujeres"] = (mujeres_df_2022["contar"]/censo_2022["Mujeres"])*1000
tasa_2022

,hombres,mujeres
0,0.0,0.000000
1,0.0,0.000000
2,0.0,0.000000
3,0.0,0.000000
4,0.0,0.000000
...,...,...
76,0.0,0.000000
77,0.0,0.000000
78,0.0,0.000000
79,0.0,0.000000
